# Creating and Evaluating Solutions <a class="anchor" id="top"></a>

In this notebook, you will train several models using Amazon Personalize, specifically: 

1. User Personalization - what items are most relevant to a specific user.
1. Similar Items - given an item, what items are similar to it.
1. Personalized Ranking - given a user and a collection of items, in what order are they most releveant.

## Outline

1. [Introduction](#intro)
1. [Create solutions](#solutions)
1. [Evaluate solutions](#eval)
1. [Using evaluation metrics](#use)
1. [Storing useful variables](#vars)

## Introduction <a class="anchor" id="intro"></a>

To recap, for the most part, the algorithms in Amazon Personalize (called recipes) look to solve different tasks, explained here:

1. **User Personalization** - New release that supports ALL HRNN workflows / user personalization needs, it will be what we use here.
1. **HRNN & HRNN-Metadata** - Recommends items based on previous user interactions with items.
1. **HRNN-Coldstart** - Recommends new items for which interaction data is not yet available.
1. **Personalized-Ranking** - Takes a collection of items and then orders them in probable order of interest using an HRNN-like approach.
1. **SIMS (Similar Items)** - Given one item, recommends other items also interacted with by users.
1. **Popularity-Count** - Recommends the most popular items, if HRNN or HRNN-Metadata do not have an answer - this is returned by default.

No matter the use case, the algorithms all share a base of learning on user-item-interaction data which is defined by 3 core attributes:

1. **UserID** - The user who interacted
1. **ItemID** - The item the user interacted with
1. **Timestamp** - The time at which the interaction occurred

We also support event types and event values defined by:

1. **Event Type** - Categorical label of an event (browse, purchased, rated, etc).
1. **Event Value** - A value corresponding to the event type that occurred. Generally speaking, we look for normalized values between 0 and 1 over the event types. For example, if there are three phases to complete a transaction (clicked, added-to-cart, and purchased), then there would be an event_value for each phase as 0.33, 0.66, and 1.0 respectfully.

The event type and event value fields are additional data which can be used to filter the data sent for training the personalization model. In this particular exercise we will not have an event type or event value. 

To run this notebook, you need to have run the previous notebooks, `01_Validating_and_Importing_User_Item_Interaction_Data` and `02_Validating_and_Importing_Item_Metadata.ipynb`, where you created a dataset and imported interaction and item metadata data into Amazon Personalize. At the end of that notebook, you saved some of the variable values, which you now need to load into this notebook.

In [1]:
%store -r

## Project constants

In [2]:
interactions_filename = "interactions.csv"
dataset_name = "movielense"
project_name = "sbcPersonalizePOC"
dataset_type = "INTERACTIONS"

## Create solutions <a class="anchor" id="solutions"></a>
[Back to top](#top)

In this notebook, you will create solutions with the following recipes:

1. User Personalization
1. SIMS
1. Personalized-Ranking

The Popularity-Count recipe is the simplest solution available in Amazon Personalize and it should only be used as a fallback, so it will also not be covered in this notebook.

Similar to the previous notebook, start by importing the relevant packages, and set up a connection to Amazon Personalize using the SDK.

In [3]:
import time
from time import sleep
import json

import boto3

In [4]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In Amazon Personalize, a specific variation of an algorithm is called a recipe. Different recipes are suitable for different situations. A trained model is called a solution, and each solution can have many versions that relate to a given volume of data when the model was trained.

To start, we will list all the recipes that are supported. This will allow you to select one and use that to build your model.

In [5]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-ecomm-customers-who-viewed-x-also-viewed',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2022, 12, 14, 15, 7, 41, 686000, tzinfo=tzlocal()),
   'domain': 'ECOMMERCE'},
  {'name': 'aws-ecomm-frequently-bought-together',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-frequently-bought-together',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2022, 12, 14, 15, 7, 41, 686000, tzinfo=tzlocal()),
   'domain': 'ECOMMERCE'},
  {'name': 'aws-ecomm-popular-items-by-purchases',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-purchases',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdat

The output is just a JSON representation of all of the algorithms mentioned in the introduction.

Next we will select specific recipes and build models with them.

### User Personalization
The User-Personalization (aws-user-personalization) recipe is optimized for all USER_PERSONALIZATION recommendation scenarios. When recommending items, it uses automatic item exploration.

With automatic exploration, Amazon Personalize automatically tests different item recommendations, learns from how users interact with these recommended items, and boosts recommendations for items that drive better engagement and conversion. This improves item discovery and engagement when you have a fast-changing catalog, or when new items, such as news articles or promotions, are more relevant to users when fresh.

You can balance how much to explore (where items with less interactions data or relevance are recommended more frequently) against how much to exploit (where recommendations are based on what we know or relevance). Amazon Personalize automatically adjusts future recommendations based on implicit user feedback.

First, select the recipe by finding the ARN in the list of recipes above.

In [6]:
user_personalization_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

#### Create the solution

First you create a solution using the recipe. Although you provide the dataset ARN in this step, the model is not yet trained. See this as an identifier instead of a trained model.

In [7]:
# already created!!
personalize.describe_solution(
    solutionArn="arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-userpersonalization")

{'solution': {'name': 'sbcPersonalizePOC-userpersonalization',
  'solutionArn': 'arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-userpersonalization',
  'performHPO': False,
  'performAutoML': False,
  'recipeArn': 'arn:aws:personalize:::recipe/aws-user-personalization',
  'datasetGroupArn': 'arn:aws:personalize:us-east-1:726011567823:dataset-group/sbcPersonalizePOC-movielense',
  'solutionConfig': {},
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2022, 12, 19, 13, 16, 48, 73000, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2022, 12, 19, 13, 16, 48, 73000, tzinfo=tzlocal()),
  'latestSolutionVersion': {'solutionVersionArn': 'arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-userpersonalization/8d48b3fa',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2022, 12, 19, 13, 16, 53, 341000, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2022, 12, 19, 13, 39, 23, 79000, tzinfo=tzlocal())

In [8]:
# user_personalization_create_solution_response = personalize.create_solution(
#     name = f"{project_name}-userpersonalization",
#     datasetGroupArn = dataset_group_arn,
#     recipeArn = user_personalization_recipe_arn
# )

user_personalization_create_solution_response = personalize.describe_solution(
    solutionArn="arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-userpersonalization")

user_personalization_solution_arn = user_personalization_create_solution_response["solution"]['solutionArn']


In [9]:
print(json.dumps(user_personalization_solution_arn, indent=2))

"arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-userpersonalization"


#### Create the solution version

Once you have a solution, you need to create a version in order to complete the model training. The training can take a while to complete, upwards of 25 minutes, and an average of 90 minutes for this recipe with our dataset. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create many models and deploy them quickly. So we will set up the while loop for all of the solutions further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [11]:
personalize.describe_solution_version(solutionVersionArn = "arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-userpersonalization/8d48b3fa")

{'solutionVersion': {'solutionVersionArn': 'arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-userpersonalization/8d48b3fa',
  'solutionArn': 'arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-userpersonalization',
  'performHPO': False,
  'performAutoML': False,
  'recipeArn': 'arn:aws:personalize:::recipe/aws-user-personalization',
  'datasetGroupArn': 'arn:aws:personalize:us-east-1:726011567823:dataset-group/sbcPersonalizePOC-movielense',
  'solutionConfig': {},
  'trainingHours': 2.445,
  'trainingMode': 'FULL',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2022, 12, 19, 13, 16, 53, 341000, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2022, 12, 19, 13, 39, 23, 79000, tzinfo=tzlocal())},
 'ResponseMetadata': {'RequestId': '2ffb515a-8c84-41e7-830b-5ffe80f4bacc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 19 Dec 2022 16:41:00 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length'

In [12]:
# userpersonalization_create_solution_version_response = personalize.create_solution_version(
#     solutionArn = user_personalization_solution_arn
# )

userpersonalization_create_solution_version_response = personalize.describe_solution_version(solutionVersionArn = "arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-userpersonalization/8d48b3fa")

In [14]:
userpersonalization_create_solution_version_response

{'solutionVersion': {'solutionVersionArn': 'arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-userpersonalization/8d48b3fa',
  'solutionArn': 'arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-userpersonalization',
  'performHPO': False,
  'performAutoML': False,
  'recipeArn': 'arn:aws:personalize:::recipe/aws-user-personalization',
  'datasetGroupArn': 'arn:aws:personalize:us-east-1:726011567823:dataset-group/sbcPersonalizePOC-movielense',
  'solutionConfig': {},
  'trainingHours': 2.445,
  'trainingMode': 'FULL',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2022, 12, 19, 13, 16, 53, 341000, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2022, 12, 19, 13, 39, 23, 79000, tzinfo=tzlocal())},
 'ResponseMetadata': {'RequestId': 'd7a80e83-2d91-4994-9958-d413e9fb7e1c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 19 Dec 2022 16:41:29 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length'

In [13]:
userpersonalization_solution_version_arn = userpersonalization_create_solution_version_response["solutionVersion"]['solutionVersionArn']
print(json.dumps(user_personalization_create_solution_response, indent=2))

TypeError: Object of type datetime is not JSON serializable

In [15]:
userpersonalization_solution_version_arn

'arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-userpersonalization/8d48b3fa'

### SIMS


SIMS is one of the oldest algorithms used within Amazon for recommendation systems. A core use case for it is when you have one item and you want to recommend items that have been interacted with in similar ways over your entire user base. This means the result is not personalized per user. Sometimes this leads to recommending mostly popular items, so there is a hyperparameter that can be tweaked which will reduce the popular items in your results. 

For our use case, using the Movielens data, let's assume we pick a particular movie. We can then use SIMS to recommend other movies based on the interaction behavior of the entire user base. The results are not personalized per user, but instead, differ depending on the movie we chose as our input.

Just like last time, we start by selecting the recipe.

In [16]:
SIMS_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

#### Create the solution

As with HRNN, start by creating the solution first. Although you provide the dataset ARN in this step, the model is not yet trained. See this as an identifier instead of a trained model.

In [18]:
sims_create_solution_response = personalize.describe_solution(
    solutionArn = "arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-sims"
)

sims_solution_arn = sims_create_solution_response["solution"]['solutionArn']
# print(json.dumps(sims_create_solution_response, indent=2))
sims_create_solution_response

{'solution': {'name': 'sbcPersonalizePOC-sims',
  'solutionArn': 'arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-sims',
  'performHPO': False,
  'performAutoML': False,
  'recipeArn': 'arn:aws:personalize:::recipe/aws-sims',
  'datasetGroupArn': 'arn:aws:personalize:us-east-1:726011567823:dataset-group/sbcPersonalizePOC-movielense',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2022, 12, 19, 13, 16, 59, 38000, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2022, 12, 19, 13, 16, 59, 38000, tzinfo=tzlocal()),
  'latestSolutionVersion': {'solutionVersionArn': 'arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-sims/0b557baa',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2022, 12, 19, 13, 17, 1, 802000, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2022, 12, 19, 13, 34, 18, 751000, tzinfo=tzlocal())}},
 'ResponseMetadata': {'RequestId': '322fb476-ef9e-403b-abfa-af246f4a4759',
  'HTT

In [12]:
# sims_create_solution_response = personalize.create_solution(
#     name = f"{project_name}-sims",
#     datasetGroupArn = dataset_group_arn,
#     recipeArn = SIMS_recipe_arn
# )

# sims_solution_arn = sims_create_solution_response['solutionArn']
# print(json.dumps(sims_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-sims",
  "ResponseMetadata": {
    "RequestId": "95d12299-5a31-43b1-9e5d-9cfcbd258131",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 19 Dec 2022 13:16:59 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "92",
      "connection": "keep-alive",
      "x-amzn-requestid": "95d12299-5a31-43b1-9e5d-9cfcbd258131"
    },
    "RetryAttempts": 0
  }
}


#### Create the solution version

Once you have a solution, you need to create a version in order to complete the model training. The training can take a while to complete, upwards of 25 minutes, and an average of 35 minutes for this recipe with our dataset. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create many models and deploy them quickly. So we will set up the while loop for all of the solutions further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [13]:
# sims_create_solution_version_response = personalize.create_solution_version(
#     solutionArn = sims_solution_arn
# )

In [14]:
# sims_solution_version_arn = sims_create_solution_version_response['solutionVersionArn']
# print(json.dumps(sims_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-sims/0b557baa",
  "ResponseMetadata": {
    "RequestId": "15f68e8b-fe86-4d29-9d32-62e15c6f7aef",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 19 Dec 2022 13:17:01 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "108",
      "connection": "keep-alive",
      "x-amzn-requestid": "15f68e8b-fe86-4d29-9d32-62e15c6f7aef"
    },
    "RetryAttempts": 0
  }
}


In [19]:
sims_solution_version_arn = "arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-sims/0b557baa"

### Personalized Ranking

Personalized Ranking is an interesting application of HRNN. Instead of just recommending what is most probable for the user in question, this algorithm takes in a user and a list of items as well. The items are then rendered back in the order of most probable relevance for the user. The use case here is for filtering on unique categories that you do not have item metadata to create a filter, or when you have a broad collection that you would like better ordered for a particular user.

For our use case, using the MovieLens data, we could imagine that a VOD application may want to create a shelf of comic book movies, or movies by a specific director. We most likely have these lists based title metadata we have. We would use personalized ranking to re-order the list of movies for each user, based on their previous tagging history. 

Just like last time, we start by selecting the recipe.

In [20]:
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

#### Create the solution

As with the previous solution, start by creating the solution first. Although you provide the dataset ARN in this step, the model is not yet trained. See this as an identifier instead of a trained model.

In [22]:
rerank_create_solution_response = personalize.describe_solution(
    solutionArn = "arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-rerank"
)

rerank_solution_arn = rerank_create_solution_response["solution"]['solutionArn']
# print(json.dumps(rerank_create_solution_response, indent=2))
rerank_solution_arn

'arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-rerank'

In [16]:
# rerank_create_solution_response = personalize.create_solution(
#     name = f"{project_name}-rerank",
#     datasetGroupArn = dataset_group_arn,
#     recipeArn = rerank_recipe_arn
# )

# rerank_solution_arn = rerank_create_solution_response['solutionArn']
# print(json.dumps(rerank_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-rerank",
  "ResponseMetadata": {
    "RequestId": "05838fb5-a79a-4d83-b11f-ca26d3bf4755",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 19 Dec 2022 13:17:06 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "94",
      "connection": "keep-alive",
      "x-amzn-requestid": "05838fb5-a79a-4d83-b11f-ca26d3bf4755"
    },
    "RetryAttempts": 0
  }
}


#### Create the solution version

Once you have a solution, you need to create a version in order to complete the model training. The training can take a while to complete, upwards of 25 minutes, and an average of 35 minutes for this recipe with our dataset. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create many models and deploy them quickly. So we will set up the while loop for all of the solutions further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [23]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

In [24]:
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-rerank/945978a6",
  "ResponseMetadata": {
    "RequestId": "2b1dba09-c832-4e83-a011-2f93a53bc542",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 19 Dec 2022 16:47:40 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "110",
      "connection": "keep-alive",
      "x-amzn-requestid": "2b1dba09-c832-4e83-a011-2f93a53bc542"
    },
    "RetryAttempts": 0
  }
}


### View solution creation status

As promised, how to view the status updates in the console:

* In another browser tab you should already have the AWS Console up from opening this notebook instance. 
* Switch to that tab and search at the top for the service `Personalize`, then go to that service page. 
* Click `View dataset groups`.
* Click the name of your dataset group, most likely something with POC in the name.
* Click `Solutions and recipes`.
* You will now see a list of all of the solutions you created above,  including a column with the status of the solution versions. Once it is `Active`, your solution is ready to be reviewed. It is also capable of being deployed.

Or simply run the cell below to keep track of the solution version creation status.

In [27]:
in_progress_solution_versions = [
#     userpersonalization_solution_version_arn, # completed
#     sims_solution_version_arn, # completed
#     rerank_solution_version_arn # skipped
]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress


KeyboardInterrupt: 

### Hyperparameter tuning

Personalize offers the option of running hyperparameter tuning when creating a solution. Because of the additional computation required to perform hyperparameter tuning, this feature is turned off by default. Therefore, the solutions we created above, will simply use the default values of the hyperparameters for each recipe. For more information about hyperparameter tuning, see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

If you have settled on the correct recipe to use, and are ready to run hyperparameter tuning, the following code shows how you would do so, using SIMS as an example.

```python
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims-hpo",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn,
    performHPO=True
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))
```

If you already know the values you want to use for a specific hyperparameter, you can also set this value when you create the solution. The code below shows how you could set the value for the `popularity_discount_factor` for the SIMS recipe.

```python
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims-set-hp",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn,
    solutionConfig = {
        'algorithmHyperParameters': {
            'popularity_discount_factor': '0.7'
        }
    }
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))
```

## Evaluate solution versions <a class="anchor" id="eval"></a>
[Back to top](#top)

It should not take more than an hour to train all the solutions from this notebook. While training is in progress, we recommend taking the time to read up on the various algorithms (recipes) and their behavior in detail. This is also a good time to consider alternatives to how the data was fed into the system and what kind of results you expect to see.

When the solutions finish creating, the next step is to obtain the evaluation metrics. Personalize calculates these metrics based on a subset of the training data. The image below illustrates how Personalize splits the data. Given 10 users, with 10 interactions each (a circle represents an interaction), the interactions are ordered from oldest to newest based on the timestamp. Personalize uses all of the interaction data from 90% of the users (blue circles) to train the solution version, and the remaining 10% for evaluation. For each of the users in the remaining 10%, 90% of their interaction data (green circles) is used as input for the call to the trained model. The remaining 10% of their data (orange circle) is compared to the output produced by the model and used to calculate the evaluation metrics.

![personalize metrics](static/imgs/personalize_metrics.png)

We recommend reading [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) to understand the metrics, but we have also copied parts of the documentation below for convenience.

You need to understand the following terms regarding evaluation in Personalize:

* *Relevant recommendation* refers to a recommendation that matches a value in the testing data for the particular user.
* *Rank* refers to the position of a recommended item in the list of recommendations. Position 1 (the top of the list) is presumed to be the most relevant to the user.
* *Query* refers to the internal equivalent of a GetRecommendations call.

The metrics produced by Personalize are:
* **coverage**: The proportion of unique recommended items from all queries out of the total number of unique items in the training data (includes both the Items and Interactions datasets).
* **mean_reciprocal_rank_at_25**: The [mean of the reciprocal ranks](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) of the first relevant recommendation out of the top 25 recommendations over all queries. This metric is appropriate if you're interested in the single highest ranked recommendation.
* **normalized_discounted_cumulative_gain_at_K**: Discounted gain assumes that recommendations lower on a list of recommendations are less relevant than higher recommendations. Therefore, each recommendation is discounted (given a lower weight) by a factor dependent on its position. To produce the [cumulative discounted gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) at K, each relevant discounted recommendation in the top K recommendations is summed together. The normalized discounted cumulative gain (NDCG) is the DCG divided by the ideal DCG such that NDCG is between 0 - 1. (The ideal DCG is where the top K recommendations are sorted by relevance.) Amazon Personalize uses a weighting factor of 1/log(1 + position), where the top of the list is position 1. This metric rewards relevant items that appear near the top of the list, because the top of a list usually draws more attention.
* **precision_at_K**: The number of relevant recommendations out of the top K recommendations divided by K. This metric rewards precise recommendation of the relevant items.

Let's take a look at the evaluation metrics for each of the solutions produced in this notebook. *Please note, your results might differ from the results described in the text of this notebook, due to the quality of the Movielens dataset.* 

### User Personalization metrics

First, retrieve the evaluation metrics for the User Personalization solution version.

In [28]:
user_personalization_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = userpersonalization_solution_version_arn
)

print(json.dumps(user_personalization_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-userpersonalization/8d48b3fa",
  "metrics": {
    "coverage": 0.0808,
    "mean_reciprocal_rank_at_25": 0.2374,
    "normalized_discounted_cumulative_gain_at_10": 0.2559,
    "normalized_discounted_cumulative_gain_at_25": 0.3028,
    "normalized_discounted_cumulative_gain_at_5": 0.2201,
    "precision_at_10": 0.0519,
    "precision_at_25": 0.0326,
    "precision_at_5": 0.0704
  },
  "ResponseMetadata": {
    "RequestId": "09a87254-46a8-4168-8407-b5af28c96ab5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 19 Dec 2022 17:04:39 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "421",
      "connection": "keep-alive",
      "x-amzn-requestid": "09a87254-46a8-4168-8407-b5af28c96ab5"
    },
    "RetryAttempts": 0
  }
}


The normalized discounted cumulative gain above tells us that at 5 items, we have less than a (38% for full 22% for small) chance in a recommendation being a part of a user's interaction history (in the hold out phase from training and validation). Around 13% of the recommended items are unique, and we have a precision of only (14% for full, 7.5% for small) in the top 5 recommended items. 

This is clearly not a great model, but keep in mind that we had to use rating data for our interactions because Movielens is an explicit dataset based on ratings. The Timestamps also were from the time that the movie was rated, not watched, so the order is not the same as the order a viewer would watch movies.

### SIMS metrics

Now, retrieve the evaluation metrics for the SIMS solution version.

In [29]:
sims_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(sims_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-sims/0b557baa",
  "metrics": {
    "coverage": 0.1575,
    "mean_reciprocal_rank_at_25": 0.134,
    "normalized_discounted_cumulative_gain_at_10": 0.154,
    "normalized_discounted_cumulative_gain_at_25": 0.2166,
    "normalized_discounted_cumulative_gain_at_5": 0.1215,
    "precision_at_10": 0.0467,
    "precision_at_25": 0.034,
    "precision_at_5": 0.0567
  },
  "ResponseMetadata": {
    "RequestId": "c67fbaf2-8e07-44b8-8790-b8d365bd725a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 19 Dec 2022 17:04:45 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "403",
      "connection": "keep-alive",
      "x-amzn-requestid": "c67fbaf2-8e07-44b8-8790-b8d365bd725a"
    },
    "RetryAttempts": 0
  }
}


In this example we are seeing a slightly elevated precision at 5 items, a little over (4.5% for full, 6.4% for small) this time. Effectively this is probably within the margin of error, but given that no effort was made to mask popularity, it may just be returning super popular results that a large volume of users have interacted with in some way. 

### Personalized ranking metrics

Now, retrieve the evaluation metrics for the personalized ranking solution version.

In [32]:
# rerank_solution_metrics_response = personalize.get_solution_metrics(
#     solutionVersionArn = rerank_solution_version_arn
# )

# print(json.dumps(rerank_solution_metrics_response, indent=2))

ResourceInUseException: An error occurred (ResourceInUseException) when calling the GetSolutionMetrics operation: Cannot get metrics for solution since it is not in ACTIVE state.

Just a quick comment on this one, here we see again a precision of near (2.7% for full, 2.2% for small), as this is based on User Personalization, that is to be expected. However the sample items are not the same for validaiton, thus the low scoring.

## Using evaluation metrics <a class="anchor" id="use"></a>
[Back to top](#top)

It is important to use evaluation metrics carefully. There are a number of factors to keep in mind.

* If there is an existing recommendation system in place, this will have influenced the user's interaction history which you use to train your new solutions. This means the evaluation metrics are biased to favor the existing solution. If you work to push the evaluation metrics to match or exceed the existing solution, you may just be pushing the User Personalization to behave like the existing solution and might not end up with something better.
* The HRNN Coldstart recipe is difficult to evaluate using the metrics produced by Amazon Personalize. The aim of the recipe is to recommend items which are new to your business. Therefore, these items will not appear in the existing user transaction data which is used to compute the evaluation metrics. As a result, HRNN Coldstart will never appear to perform better than the other recipes, when compared on the evaluation metrics alone. Note: The User Personalization recipe also includes improved cold start functionality

Keeping in mind these factors, the evaluation metrics produced by Personalize are generally useful for two cases:
1. Comparing the performance of solution versions trained on the same recipe, but with different values for the hyperparameters and features (impression data etc)
1. Comparing the performance of solution versions trained on different recipes (except HRNN Coldstart).

Properly evaluating a recommendation system is always best done through A/B testing while measuring actual business outcomes. Since recommendations generated by a system usually influence the user behavior which it is based on, it is better to run small experiments and apply A/B testing for longer periods of time. Over time, the bias from the existing model will fade.

## Storing useful variables <a class="anchor" id="vars"></a>
[Back to top](#top)

Before exiting this notebook, run the following cells to save the version ARNs for use in the next notebook.

In [33]:
import os
from utils import save_json

os.makedirs("results", exist_ok=True)

data = {
"userpersonalization_solution_version_arn": userpersonalization_solution_version_arn,
"sims_solution_version_arn": sims_solution_version_arn,
"rerank_solution_version_arn": rerank_solution_version_arn,
"user_personalization_solution_arn": user_personalization_solution_arn,
"sims_solution_arn": sims_solution_arn,
# "rerank_solution_arn": rerank_solution_arn
}

save_json("results/03.json", data=data)

json file saved at: results/03.json


In [34]:
%store userpersonalization_solution_version_arn
%store sims_solution_version_arn
%store rerank_solution_version_arn
%store user_personalization_solution_arn
%store sims_solution_arn
%store rerank_solution_arn

Stored 'userpersonalization_solution_version_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'rerank_solution_version_arn' (str)
Stored 'user_personalization_solution_arn' (str)
Stored 'sims_solution_arn' (str)
Stored 'rerank_solution_arn' (str)


In [35]:
json_data = dict(globals().copy())
all_arns = dict()

for key, val in json_data.items():
    if ("arn" in key):
        print(f"{key}: {val}\n")
        all_arns[key] = val
        
save_json("results/03_all_arns.json", data=all_arns)

interactions_dataset_arn: arn:aws:personalize:us-east-1:726011567823:dataset/sbcPersonalizePOC-movielense/INTERACTIONS

dataset_group_arn: arn:aws:personalize:us-east-1:726011567823:dataset-group/sbcPersonalizePOC-movielense

role_arn: arn:aws:iam::726011567823:role/sbcPersonalizePOCRolePOC

interaction_schema_arn: arn:aws:personalize:us-east-1:726011567823:schema/sbcPersonalizePOC-movielense-interactions

items_dataset_arn: arn:aws:personalize:us-east-1:726011567823:dataset/sbcPersonalizePOC-movielense/ITEMS

itemmetadataschema_arn: arn:aws:personalize:us-east-1:726011567823:schema/sbcPersonalizePOC-movielense-item

userpersonalization_solution_version_arn: arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-userpersonalization/8d48b3fa

sims_solution_version_arn: arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-sims/0b557baa

rerank_solution_version_arn: arn:aws:personalize:us-east-1:726011567823:solution/sbcPersonalizePOC-rerank/945978a6

user_

In [36]:
with open("results/03_all.txt", "w+") as f:
    f.write(str(json_data))